In [ ]:
import pandas as pd
import pyodbc as pdb
import psycopg2
import numpy as np
import faker

### CONEXIONES

In [ ]:
host = "localhost"
database = "postgres"
user = "root"
password = "root"
port = "5432"

In [ ]:
connection_target = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )

# Cursor
cur_target = connection_target.cursor()

In [ ]:
query = """
SELECT *
FROM jubilados
"""

cur_target.execute(query)
rows = cur_target.fetchall()

list_of_rows = []

# Para guardar todas las filas
for row in rows:
    list_of_rows.append(row)

# Obtener los nombres de las columnas desde la descripción del cursor
column_names = [desc[0] for desc in cur_target.description]

# Crea el DataFrame con los datos y los nombres de las columnas
df = pd.DataFrame(list_of_rows, columns=column_names)
display(df)
df.dtypes

In [ ]:
query = """
SELECT *
FROM viajes
"""

cur_target.execute(query)
rows = cur_target.fetchall()

list_of_rows = []

#Para guardar todas las filas
for row in rows:
    list_of_rows.append(row)

# Obtener los nombres de las columnas desde la descripción del cursor
column_names = [desc[0] for desc in cur_target.description]

# Crea el DataFrame con los datos y los nombres de las columnas
df_viajes = pd.DataFrame(list_of_rows, columns=column_names)
display(df_viajes)

In [ ]:
fila_aleatoria = df_viajes.sample(n=1)

# Obtiene el valor de geografico_id del viaje elegido al azar
geografico_id_aleatorio = fila_aleatoria['geografico_id'].values[0]
info_viaje = fila_aleatoria.to_dict(orient='records')[0] #guarda la info del viaje en un diccionario

# Filtra df basándose en geografico_id, de forma que puedan acceder los no viven en ese sitio
df_filtrado = df[df['geografico_id'] != geografico_id_aleatorio]

### FILTROS

In [ ]:
# PUNTOS INICIALES
df_filtrado['puntos'] = 0
df_filtrado['puntos'] = df_filtrado['puntos'].astype(float)  # Para insertar decimales

In [ ]:
#PUNTOS POR EDAD
df_filtrado['puntos'] += np.minimum(10, 10 * (np.exp(0.01 * (df_filtrado['edad'] - 58)) ** 1.15 - 1) / (np.exp(0.01 * (90 - 58)) ** 1.15 - 1)+ 0.6)
#np.minimum 10 es para que el máximo sea 10, que de igual si tiene 90 o 2332478 años, los puntos van a ser 10

In [ ]:
#PUNTOS POR ENDEUDAMIENTO
df_filtrado.loc[df_filtrado['endeudamiento'] == True, 'puntos'] -= 7

In [ ]:
#PUNTOS POR AÑOS TRIBUTADOS
df_filtrado['puntos'] += ((df_filtrado['años_tributados'] - 15) * 0.37)

In [ ]:
#PUNTOS POR FUMADOR
df_filtrado.loc[df_filtrado['fumador'] == True, 'puntos'] -=0.5

In [ ]:
#PUNTOS POR NUMERO DE PROPIEDADES
df_filtrado['puntos'] -= (df_filtrado['numero_propiedades']* 1)

In [ ]:
#PUNTOS POR PARTICIPACION VOLUNTARIADO
df_filtrado.loc[df_filtrado['participacion_voluntario'] == True, 'puntos'] += 2

In [ ]:
#PUNTOS POR NACIONALIDAD
df_filtrado.loc[df_filtrado['nacionalidad_española'] == True, 'puntos' ] += 0.1

In [ ]:
#CAPACIDAD VIAJE
df_sorted = df_filtrado.sort_values(by='puntos', ascending=False) #para ordenar de más puntos a menos
capacidad_viaje = info_viaje['capacidad_viaje'] #crear una variable con la capacidad del viaje a partir del diccionario anterior

# RESULTADO
abuelos_seleccionados = df_sorted.head(capacidad_viaje) #limitamos por capacidad y mostramos
viaje_id = info_viaje['viaje_id']
nombre_archivo = f'ganadores_viaje{viaje_id}.csv'
print(*info_viaje.items(), sep='\n')
display(abuelos_seleccionados)
abuelos_seleccionados.to_csv(nombre_archivo, index=False)
